In [1]:
!pip install awswrangler

     |████████████████████████████████| 179 kB 24.3 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 2.4 MB/s  eta 0:00:01
     |████████████████████████████████| 43 kB 412 kB/s  eta 0:00:01


In [2]:
import pandas as pd
import awswrangler as wr
import sagemaker
bucket = sagemaker.Session().default_bucket()
bucket ## bucket asignado al equipo acorde a nuestro SSO 

'sagemaker-us-east-1-058528764918'

In [3]:
sess = sagemaker.session.Session(default_bucket= 's3://{}'.format(bucket))  ##seteado de bucket a usar
role = sagemaker.get_execution_role()   ## levantamiento de roles

### >> LOAD DATA

In [4]:
df = wr.s3.read_csv('s3://{}/muestra.csv'.format(bucket), sep=';')
df.head()

,TIP_DOC,NUM_DOC,COD_UNICO,PERIODO,MTO_DEUDA_COL_DIR_IBK,MTO_DEUDA_COL_DIR_SF,TRG_1M,TRG_2M,TRG_3M,TRG_4M,...,FLG_2M_ANTES,FLG_3M_ANTES,FLG_4M_ANTES,AUMENTOS_BPE_1M_ANTES,AUMENTOS_BPE_2M_ANTES,AUMENTOS_BPE_3M_ANTES,AUMENTOS_BPE_4M_ANTES,GENERO_F,GENERO_M,EDAD_RRLL
0,2,20453973787,NaN,202006,0.0,9617.80,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,NaN
1,2,20453974244,10743339.0,202009,0.0,435514.55,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,NaN
2,2,20453981534,NaN,202101,0.0,22579.21,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,NaN
3,2,20453981534,NaN,202102,0.0,19682.35,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,NaN
4,2,20453984398,NaN,202004,0.0,315041.57,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [5]:
df['PERIODO'].value_counts().sort_index(ascending=False)

202103    4726
202102    4720
202101    4661
202012    4064
202011    4161
202010    4083
202009    4103
202008    3725
202007    3379
202006    3284
202005    3251
202004    3026
202003    3017
202002    3066
202001    3028
201912    3086
201911    3134
201910    3032
201909    3045
201908    3103
201907    3075
201906    3227
201905    3066
201904    3071
201903    3160
201902    3126
201901    3158
201812    3197
201712    3226
Name: PERIODO, dtype: int64

### >> CREACION DE VARIABLES

In [6]:
# Variaciones
df['MTO_DEUDA_COL_DIR_VIG_SF_U3M_U12M_MAX'] = df['MTO_DEUDA_COL_DIR_VIG_SF_U3M_MAX']/df['MTO_DEUDA_COL_DIR_VIG_SF_U6M_MAX']
df['MTO_DEUDA_COL_DIR_VIG_SF_U6M_U12M_MAX'] = df['MTO_DEUDA_COL_DIR_VIG_SF_U6M_MAX']/df['MTO_DEUDA_COL_DIR_VIG_SF_U12M_MAX']
df['MTO_DEUDA_COL_DIR_VIG_SF_U3M_U6M_MAX'] = df['MTO_DEUDA_COL_DIR_VIG_SF_U3M_MAX']/df['MTO_DEUDA_COL_DIR_VIG_SF_U6M_MAX']

df['MTO_DEUDA_COL_DIR_VIG_SF_U3M_U12M_AVG'] = df['MTO_DEUDA_COL_DIR_VIG_SF_U3M_AVG']/df['MTO_DEUDA_COL_DIR_VIG_SF_U6M_AVG']
df['MTO_DEUDA_COL_DIR_VIG_SF_U6M_U12M_AVG'] = df['MTO_DEUDA_COL_DIR_VIG_SF_U6M_AVG']/df['MTO_DEUDA_COL_DIR_VIG_SF_U12M_AVG']
df['MTO_DEUDA_COL_DIR_VIG_SF_U3M_U6M_AVG'] = df['MTO_DEUDA_COL_DIR_VIG_SF_U3M_AVG']/df['MTO_DEUDA_COL_DIR_VIG_SF_U6M_AVG']

df['CLAS_SBS_U3_U6_MAX_SF'] = df['CLAS_SBS_U3_MAX_SF']/df['CLAS_SBS_U6_MAX_SF']
df['CLAS_SBS_U3_U12_MAX_SF'] = df['CLAS_SBS_U3_MAX_SF']/df['CLAS_SBS_U12_MAX_SF']
df['CLAS_SBS_U6_U12_MAX_SF'] = df['CLAS_SBS_U6_MAX_SF']/df['CLAS_SBS_U12_MAX_SF']

In [7]:
try:
    df.describe(include=[object]).columns
except Exception as e:
    print(str(e))

No objects to concatenate


### >> SEPARATE TRAIN / TEST

In [8]:
df['PERIODO'] = df['PERIODO'].astype(str)

meses_test = ['202012','202011']
meses_entrenamiento = ['202010', '202009', '202008', '202007','202006',
                       '202005', '202004', '202003', '202002', '202001',
                       '201912', '201911', '201910', '201909', '201908']

In [9]:
valid = df[df['PERIODO'].isin(meses_test)]
train = df[df['PERIODO'].isin(meses_entrenamiento)]

del df
valid.shape, train.shape

((8225, 64), (49362, 64))

In [10]:
y_train = train['TRG_3M'].copy()
del train['TRG_3M']

y_valid = valid['TRG_3M'].copy()
del valid['TRG_3M']

### >> GUARDADO S3

In [11]:
prefix = 'vpc'     # subcarpeta de guardado
output_path = 's3://{}/{}/victor'.format(bucket, prefix)
output_path

's3://sagemaker-us-east-1-058528764918/vpc/victor'

In [12]:
# definir rutas de datos dentro de bucket
train_fname = "{}/train.csv".format(output_path)
print(train_fname)

val_fname = "{}/validation_set.csv".format(output_path)
print(val_fname)

s3://sagemaker-us-east-1-058528764918/vpc/victor/train.csv
s3://sagemaker-us-east-1-058528764918/vpc/victor/validation_set.csv


In [13]:
train.to_csv(
    's3://sagemaker-us-east-1-058528764918/vpc/victor/validation_set_sin_target.csv', 
    header=False, index=False
)

In [ ]:
pd.concat(
    [y_train, train],axis=1, ignore_index=True   # xgboost por defecto te pdide que el target esté en tu 1era columna
).to_csv(
    train_fname, 
    header=False, index=False
)

pd.concat(
    [y_valid, valid], axis=1, ignore_index=True # xgboost por defecto te pdide que el target esté en tu 1era columna
).to_csv(
    val_fname, 
    header=False, index=False
)

### >> VALIDAICON DE GUARDADO

In [ ]:
train = wr.s3.read_csv(train_fname, sep=',')
train.shape

In [ ]:
valid = wr.s3.read_csv(val_fname, sep=',')
valid.shape